Updating Schools Layers

1.	Updating individual school layer in school database(school.gdb)
2.	Merge schools into four categories and create four layers : 
    o	Elem
    o	Midd
    o	High
    o	Other
This step can be done through running python scripts below.Make sure change output file name since it cannot exist before merge.


In [ ]:
import arcpy  #you can copy the code to arcgis python window
elem =[]
mid =[]
high =[]
other =[]

loc = "/Projects/UrbanSim/NEW_DIRECTORY/Schools/school.gdb"
#Make sure change output file name since it cannot exist before merge

arcpy.env.workspace = loc + "/elem"
for FeatClass in arcpy.ListFeatureClasses():
        elem.append(FeatClass)

arcpy.Merge_management(elem, loc+"/Merged_elem")


arcpy.env.workspace =loc + "/midd"
for FeatClass in arcpy.ListFeatureClasses():
        mid.append(FeatClass)

arcpy.Merge_management(mid, loc+"/Merged_midd")

arcpy.env.workspace= loc + "/high"
for FeatClass in arcpy.ListFeatureClasses():
        high.append(FeatClass)
arcpy.Merge_management(high, loc+"/Merged_high")


arcpy.env.workspace= loc + "/Other"
for FeatClass in arcpy.ListFeatureClasses():
        other.append(FeatClass)        
arcpy.Merge_management(other, loc+"/Merged_other")        

Joining School data to parcel data

3.	Add target parcel data into database, and conduct spatial join with the merged school layers using scripts below

In [ ]:
target = loc + "/prcl15"
join_features1 =loc+"/Merged_elem"
join_features2 = loc+"/Merged_mid"
join_features3 = loc+"/Merged_high"
join_features4 = loc+"/Merged_other"

output1 = loc+"output1"
output2 = loc+"output2"
output3 = loc+"output3"
output4 = loc+"output4"

arcpy.SpatialJoin_analysis(target, join_features1,output1,"JOIN_ONE_TO_ONE","#","#","INTERSECT")
arcpy.SpatialJoin_analysis(output1, join_features2,output2,"JOIN_ONE_TO_ONE","#","#","INTERSECT")
arcpy.SpatialJoin_analysis(output2, join_features3,output3,"JOIN_ONE_TO_ONE","#","#","INTERSECT")
arcpy.SpatialJoin_analysis(output3, join_features4,output4,"JOIN_ONE_TO_ONE","#","#","INTERSECT")

4.	Exported parcel with joint information to “output.shp” file
5.	Combine fields together in python script below

In [9]:
import sys, os
import csv
import pandas as pd
import numpy as np
from dbfread import DBF
os.chdir("/Projects/UrbanSim/NEW_DIRECTORY/Schools/2014parcel/shp")

In [4]:
#read dbf into dataframe
from simpledbf import Dbf5
dbf = Dbf5('prcl15_school.dbf')
df = dbf.to_dataframe()

In [17]:
#conbine join
df['ElemName'] =df['Elem_Name']
df['MiddName'] =df['Midd_Name']
df['HighName'] =df['High_Name']
df['ElemName'] = np.where(pd.isnull(df['Midd_Name']), df['Midd_Name_'],df['Midd_Name'])
df['MiddName'] = np.where(pd.isnull(df['Elem_Name']), df['Elem_Name_'],df['Elem_Name'])
df['HighName'] = np.where(pd.isnull(df['High_Name']), df['High_Name_'],df['High_Name'])

In [18]:
df['ElemID'] = df['Elem_ID']
df['MiddID'] = df['Midd_ID']
df['HighID'] = df['High_ID']
df['ElemID'] = np.where(df['Elem_ID']==0, df['Elem_ID_1'],df['Elem_ID'])
df['MiddID'] = np.where(df['Midd_ID']==0, df['Midd_ID_1'],df['Midd_ID'])
df['HighID'] = np.where(df['High_ID']==0, df['High_ID_1'],df['High_ID'])

In [27]:
#export dataframe to csv file
col_want = ['PIN','PSRC_ID','POINT_X','POINT_Y','ElemID','ElemName',
            'MiddID','MiddName','HighID','HighName']
output = df[col_want]
output.to_csv('/Projects/UrbanSim/NEW_DIRECTORY/Schools/2014parcel/prcl_sch_0308.csv')

In [28]:
output.count()

PIN         1268946
PSRC_ID     1276388
POINT_X     1276388
POINT_Y     1276388
ElemID      1276388
ElemName    1196860
MiddID      1276388
MiddName    1196860
HighID      1276388
HighName    1201751
dtype: int64